In [2]:
import numpy as np
import re
import json
import random
import sys
sys.path.append("../../")
from yelp_sentiment_attention.train import *
from yelp_multiclass.data.yelp_dataset import load_word_indices

from yelp_multiclass.data.textProcess import createTextSent,tokenizeText,buildIndexToWordDict

c:\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


2018-07-13 22:03:07,973 - Yelp_Review_Processor - INFO - Looking for data :: G:\work\nlp\datasets\yelp\attention\dataset\yelp_review_binary_processed.npz
2018-07-13 22:03:07,975 - Yelp_Review_Processor - INFO - Loading data from cache
2018-07-13 22:03:10,767 - Yelp_Review_Processor - INFO - Loaded data in cache


## Restore trained model from file

In [3]:
saver = tf.train.Saver()

log.info('Restorng saved model')

with tf.Session() as sess:
    saver.restore(sess, MODEL_PATH)
    x_batch_test, y_batch_test = X_test[:550], y_test[:550]
    seq_len_test = np.array([list(x).index(0) + 1 for x in x_batch_test])
    results = sess.run([alphas,y_hat], feed_dict={batch_ph: x_batch_test, target_ph: y_batch_test,
                                                seq_len_ph: seq_len_test, keep_prob_ph: 1.0})

2018-07-13 22:03:12,443 - Yelp_Review_Processor - INFO - Restorng saved model
INFO:tensorflow:Restoring parameters from G:\work\nlp\datasets\yelp\attention\model\
2018-07-13 22:03:13,208 - tensorflow - INFO - Restoring parameters from G:\work\nlp\datasets\yelp\attention\model\


## Read Vocab from disk

In [4]:
log.info('Reading Vocab')
word_index = load_word_indices()
index_word = buildIndexToWordDict(word_index)
log.info('Done')

2018-07-13 22:03:28,414 - Yelp_Review_Processor - INFO - Reading Vocab
2018-07-13 22:03:30,095 - Yelp_Review_Processor - INFO - Done


## Read results from model and generate file

In [5]:
alphas_values = results[0][0]
filename="visualization2.html"
with open(filename, "w") as html_file:
    for X,alpha_values,Y_hat,Y in zip(x_batch_test,results[0],results[1],y_batch_test ) :
        html_file.write("<br/><hr/>");
        sample = [x - 1 for x in X]
        sample[0] = 1
        sample=[0 if x == -1 else x for x in sample]
        words = list(map(index_word.get, sample))
        html_file.write('%s  %s\n' % (Y, Y_hat));
        if Y_hat > 0:
            div_color='#b8efc8'
        else:
            div_color = '#ff848a'
        html_file.write('<div style="background-color:%s">\n' % div_color);
        for word, alpha in zip(words, alphas_values / alphas_values.max()):
            if word == ":START:":
                continue
            elif word == ":PAD:":
                break
            html_file.write('<font style="background: rgba(255, 255, 0, %f)">%s</font>\n' % (alpha, word))
        html_file.write('</div>\n');
print('Working director is ' +  os.getcwd())

Working director is K:\python_projects\KerasTextClassifier\yelp_sentiment_attention\test
